In [1]:
import numpy as np
import matplotlib.pyplot as plt
import random
from math import cos, sin
import matplotlib.animation as animation

random.seed(1)

plt.rcParams["figure.figsize"] = (10, 10)

%matplotlib notebook

In [2]:
# Hyper-parameters
ENERGY_LOSS_GENERAL = 0.1
ENERGY_LOSS_ACTIONS = 0.2

FOOD_TO_ENERGY = 0.1
WATER_TO_ENERGY = 0.1

# Rotation vectors for movement
theta = np.deg2rad(45)
rot_right = np.array([[cos(theta), -sin(theta)], [sin(theta), cos(theta)]])
rot_left = np.array([[cos(-theta), -sin(-theta)], [sin(-theta), cos(-theta)]])

class Being:
    """
    Living being representation. If energy reaches 0 it will be dead

    Energy goes down when performing actions, low energy leads to lower happiness.
    """
    # Subjective states (things the "brain" feels)
    happiness = 1
    hunger = 0
    thirst = 0

    # Objective states (hidden from the "brain")
    food = 1
    water = 1
    energy = 1

    direction = [1, 0] # vector from (0,0) (the being) to the direction its facing
    action_space = ['NOOP', 'TURN_LEFT', 'TURN_RIGHT', 'MOVE', 'EAT', 'DRINK']

    def choose_action(self):
        action = random.choice(self.action_space)

        if action != 'NOOP':
            self.energy -= ENERGY_LOSS_ACTIONS

        if action == 'TURN_LEFT' or action == 'TURN_RIGHT':
            rot = rot_left if action == 'TURN_LEFT' else rot_right
            self.direction = np.round(np.dot(rot, self.direction), 0).astype(int)

        return action

    def step(self):
        self.energy -= ENERGY_LOSS_GENERAL

        if self.food > 0:
            self.food -= FOOD_TO_ENERGY
            self.energy += FOOD_TO_ENERGY

        if self.water > 0:
            self.water -= WATER_TO_ENERGY
            self.energy += WATER_TO_ENERGY

    def color(self):
        return 155 + self.energy * 100

class Cell:
    def __init__(self, x, y):
        self.x = x
        self.y = y
        self.type = 'NONE'
        self.content = None

    def update(self, type, content=None):
        self.type = type
        self.content = content

    def color(self):
        if self.type == 'NONE':
            return 0

        if self.type == 'BEING':
            return self.content.color()

class World:
    def __init__(self, w=128, h=128):
        self.w = w
        self.h = h
        self.state = np.empty((w, h), dtype=object)
        for i in range(w):
            for j in range(h):
                self.state[i, j] = Cell(i, j)

        self.im = plt.imshow(np.zeros((self.w, self.h)), cmap='gray', vmin=0, vmax=255, animated=True)

    def step(self):
        state = np.copy(self.state)

        for i, row in enumerate(self.state):
            for j, cell in enumerate(row):
                if cell.type != 'BEING':
                    continue

                cell.content.step()
                action = cell.content.choose_action()

                if action == 'MOVE':
                    # lets see if the desired cell is empty
                    direction = cell.content.direction
                    next_loc = [
                        max(0, min(self.w - 1, cell.x + direction[0])),
                        max(0, min(self.h - 1, cell.y + direction[1]))
                    ]

                    if state[next_loc[0], next_loc[1]].type == 'NONE':
                        # cell is empty, lets move!
                        state[next_loc[0], next_loc[1]].update('BEING', cell.content)
                        state[i, j].update('NONE')

        self.state = state

    def spawn(self, number):
        for _ in range(number):
            x = random.randint(0, self.w - 1)
            y = random.randint(0, self.h - 1)

            while self.state[x, y].type != 'NONE':
                x = random.randint(0, self.w - 1)
                y = random.randint(0, self.h - 1)
                # TODO: this could be infinite

            being = Being()
            self.state[x, y].update('BEING', being)

    def render(self):
        state = np.zeros((self.w, self.h))
        for i, row in enumerate(self.state):
            for j, cell in enumerate(row):
                state[i, j] = cell.color()

        self.im.set_array(state)
        return [self.im]

fig = plt.figure(figsize=(10, 10))
plt.xticks([])
plt.yticks([])

world = World()
world.spawn(10)
world.render()

def animate():
    world.step()
    return world.render()

fps = 30

animation.FuncAnimation(fig, animate, frames=100, interval=1000/fps)
plt.show()

<IPython.core.display.Javascript object>

/opt/homebrew/Caskroom/miniforge/base/envs/evolving/lib/python3.8/site-packages/matplotlib/animation.py:887: UserWarning: Animation was deleted without rendering anything. This is most likely not intended. To prevent deletion, assign the Animation to a variable, e.g. `anim`, that exists until you have outputted the Animation using `plt.show()` or `anim.save()`.
  warnings.warn(


In [20]:
from math import cos, sin

theta = np.deg2rad(45)
rot_right = np.array([[cos(theta), -sin(theta)], [sin(theta), cos(theta)]])
rot_left = np.array([[cos(-theta), -sin(-theta)], [sin(-theta), cos(-theta)]])

vec = np.array([0, -1])

print(vec)
for i in range(9):
    vec = np.round(np.dot(rot_left, vec), 0).astype(int)
    print(vec)

[ 0 -1]
[-1 -1]
[-1  0]
[-1  1]
[0 1]
[1 1]
[1 0]
[ 1 -1]
[ 0 -1]
[-1 -1]


In [5]:
import arcade

SCREEN_WIDTH = 1000
SCREEN_HEIGHT = 650
SCREEN_TITLE = "Platformer"


class MyGame(arcade.Window):
    """
    Main application class.
    """

    def __init__(self):

        # Call the parent class and set up the window
        super().__init__(SCREEN_WIDTH, SCREEN_HEIGHT, SCREEN_TITLE)

        arcade.set_background_color(arcade.csscolor.CORNFLOWER_BLUE)

    def setup(self):
        """Set up the game here. Call this function to restart the game."""
        pass

    def on_draw(self):
        """Render the screen."""

        self.clear()
        # Code to draw the screen goes here


window = MyGame()
window.setup()
arcade.run()


KeyboardInterrupt: 

In [ ]:
window.close()
arcade.exit()